# Backend-Agnostic Two Moons

This example notebook covers a backend-agnostic model trained online on the two moons dataset. You will learn how to:

1. Use BayesFlow with your backend of choice
2. Define joint distributions with BayesFlow decorators
3. Fit an amortized posterior with the new BayesFlow interface

## 1. Select a Backend

You can select a backend by setting the KERAS_BACKEND environment variable to one of "jax", "tensorflow", or "torch". You can do this by running any of the following commands. For this notebook, we set the variable dynamically, so you can switch it around as you like, but in general we recommend the conda environment version.

1. Using your system environment variables:
```
export KERAS_BACKEND="torch"
```

2. Using conda:
```
conda env config vars set KERAS_BACKEND="torch"
```

3. Dynamically in Python:

In [20]:
import os
# "jax", "tensorflow", or "torch"
os.environ["KERAS_BACKEND"] = "torch"

## 2. Defining the Simulation

We will use online training on the two moons toy dataset in this example. To define a joint distribution, we use convenience decorators:

In [21]:
import keras

import numpy as np

import bayesflow.experimental as bf

Context Distribution:

In [22]:
@bf.distribution
def two_moons_context():
    # r ~ N(0.1, 0.01)
    r = keras.random.normal(shape=(1,), mean=0.1, stddev=0.01)
    # alpha ~ U(-π/2, π/2)
    alpha = keras.random.uniform(shape=(1,), minval=-0.5 * np.pi, maxval=0.5 * np.pi)
    return dict(r=r, alpha=alpha)

Parameter Prior:

In [23]:
@bf.distribution
def two_moons_prior():
    # θ ~ U(-1, 1)
    theta = keras.random.uniform(shape=(2,), minval=-1.0, maxval=1.0)
    return dict(theta=theta)

Simulator:

In [24]:
@bf.distribution
def two_moons_likelihood(r, alpha, theta):
    # simulate the two moons
    x1 = -keras.ops.abs(theta[0] + theta[1]) / np.sqrt(2.0) + r * keras.ops.cos(alpha) + 0.25
    x2 = (-theta[0] + theta[1]) / np.sqrt(2.0) + r * keras.ops.sin(alpha)
    return dict(x=keras.ops.concatenate([x1, x2], axis=0))

Combining these to yield a joint distribution:

In [25]:
joint_distribution = bf.simulation.DefaultJointDistribution(
    local_context=two_moons_context,
    global_context=None,
    prior=two_moons_prior,
    likelihood=two_moons_likelihood,
)

## 3. Defining the training strategy via a Dataset

We want to train online, meaning we sample new data for each training step. BayesFlow already provides a Dataset for such common cases:

In [26]:
# pass batch size and steps per epoch here for now
# support this issue to fix that and move these to posterior.fit()
# https://github.com/keras-team/keras/issues/19528
train_dataset = bf.datasets.OnlineDataset(
    joint_distribution=joint_distribution,
    batch_size=64,
    workers=8,
    # use_multiprocessing=True,
)
validation_dataset = bf.datasets.OnlineDataset(
    joint_distribution=joint_distribution,
    batch_size=64,
    workers=8,
    # use_multiprocessing=True,
)

## 4. Defining Summary and Inference Networks

We do not want to use a summary network for this example, so we just leave this blank. As the inference network, we use a 4-layer coupling flow with affine transforms.

In [27]:
summary_network = None

In [28]:
# define this manually, for now
class Subnet(keras.Layer):
    def __init__(self, out_features):
        super().__init__()
        self.out_features = out_features
        
        self.network = keras.Sequential([
            keras.layers.Dense(512, activation="relu"),
            keras.layers.Dense(2 * out_features),
        ])
        
    def call(self, x):
        return self.network(x)

In [29]:
# use a sequential coupling flow
# method name is subject to change
# we will allow to use the default BayesFlow networks in the future
inference_network = bf.networks.CouplingFlow.uniform(
    subnet_constructor=Subnet,
    # 2 parameters
    target_dim=2,
    num_layers=4,
    transform="affine",
    base_distribution="normal",
)

## 5. Putting Things Together

Now that the model internals are defined, collect them in an `AmortizedPosterior` and train.

In [30]:
posterior = bf.AmortizedPosterior(inference_network=inference_network, summary_network=summary_network)

In [31]:
optimizer = keras.optimizers.AdamW(learning_rate=1e-3, weight_decay=0.01)

In [32]:
posterior.compile(optimizer)

In [33]:
callbacks = [
    # track losses and metrics in TensorBoard
    keras.callbacks.TensorBoard("logs/two_moons/"),
    # save the best model each epoch
    keras.callbacks.ModelCheckpoint("logs/two_moons/checkpoints/model.keras", save_best_only=True)
]

Finally, fit your model:

In [34]:
posterior.fit(train_dataset, validation_data=validation_dataset, epochs=100, callbacks=callbacks)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/lars/miniforge3/envs/keras-torch/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3577, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_6650/1252914424.py", line 1, in <module>
    posterior.fit(train_dataset, validation_data=validation_dataset, epochs=100, callbacks=callbacks)
  File "/home/lars/miniforge3/envs/keras-torch/lib/python3.11/site-packages/keras/src/utils/traceback_utils.py", line 122, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "/home/lars/miniforge3/envs/keras-torch/lib/python3.11/site-packages/torch/utils/data/dataloader.py", line 631, in __next__
    data = self._next_data()
           ^^^^^^^^^^^^^^^^^
  File "/home/lars/miniforge3/envs/keras-torch/lib/python3.11/site-packages/torch/utils/data/dataloader.py", line 675, in _next_data
    data = self._dataset_fetcher.fetch(index)  # may raise StopIteration
           ^^^^^^^^^^^^^^^

In [ ]:
bf.diagnostics.show_posterior(posterior=posterior)